## Лабораторна робота №2: "Імплементація алгоритмів стиснення"

Склад команди та розподіл виконаних завдань:

-
-

Для кожного з алгоритмів поданих нижче
- опишіть як працює алгорит
- напишіть класи з методами encode та decode
- перевірте правильність кодування та декодування
- дослідіть час виконання коду в залежності від розмірів вхідних даних
- оцініть ступінь стиснення(у відсотка) в залежності від розмірів
- напишіть висновок про ефективність різних алгоритмів та умови за яких той чи інший алгоритм дають кращий результат

# Алгоритм Гаффмана

В цьому алгоритмі доцільно імплементувати клас node та додаткові функції в Huffman для побудови дерева кодування

In [ ]:
class Huffman:
    def encode(self, text: str) -> tuple[str, dict[str, str]]:
        pass

    def decode(self, code: str, coding_dict: dict[str, str]):
        pass

# Алгоритм LZW

In [ ]:
class LZW:
    def encode(self, text: str) -> tuple[str, list]:
        pass

    def decode(self, code: str, coding_dict: list) -> str:
        pass

# Алгоритм LZ77

Потрібно заміряти розміри саме тексту, проте для роботи доцільно використовувати список тюплів, тому для зручності варто імплементувати додаткові алгоритми _text2list та _list2text

In [2]:
class LZ77:
    def __init__(self, buffer_size: int):
        self.buf_size = buffer_size

    def encode(self, text: str) -> str:
        result = []
        cursor = 0
        while cursor < len(text):
            offset = -1
            copy_len = -1
            matched = ''
            for length in range(1, min(len(text)-cursor, self.buf_size)):
                entry = text[cursor : cursor+length]
                current_offset = text.rfind(entry, max(0, cursor-self.buf_size), cursor)
                if current_offset != -1 and length > copy_len:
                    offset = cursor - current_offset
                    copy_len = length
                    matched = entry
            if matched:
                result.append((offset, copy_len, text[cursor + copy_len]))
                cursor += copy_len + 1
            else:
                result.append((0, 0, text[cursor]))
                cursor += 1
        return self._list2text(result)

    def decode(self, code: str) -> str:
        code = self._text2list(code)
        result = ''
        cursor = 0
        while cursor < len(code):
            offset, length, end = code[cursor]
            if offset == 0:
                result += end
                cursor += 1
            else:
                start = len(result) - offset
                for i in range(length):
                    result += result[start + i]
                result += end
                cursor += 1
        return result

    def _text2list(self, string:str):
        return [(int(el.split(',')[0]), int(el.split(',')[1]), el.split(',')[2]) for el in string.split('|')]

    def _list2text(self, lst: list):
        return '|'.join([','.join([str(s) for s in el]) for el in lst])



lz77 = LZ77(5)
print((res:=lz77.encode((text:='abcabacaaacbacabaacaacba'))))
print('-----------------------')
print(lz77.decode(res))
print(text == lz77.decode(res))

0,0,a|0,0,b|0,0,c|3,2,a|4,2,a|4,2,b|3,2,a|4,2,a|5,2,a|3,1,b|0,0,a
-----------------------
abcabacaaacbacabaacaacba
True


# Алгоритм Deflate

In [ ]:
class Deflate:
    def __init__(self, buffer_size: int):
        pass

    def encode(self, text: str) -> str:
        pass

    def decode(self, code: str) -> str:
        pass